In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import scipy.io as sio
import csv

In [ ]:
mat_contents = sio.loadmat(file_name='MSdata.mat')
mat_contents

In [ ]:
test_x = mat_contents['testx']
train_x = mat_contents['trainx']
train_y = mat_contents['trainy']

In [ ]:
# OLS (there are no penalty terms)
reg = linear_model.LinearRegression()
reg.fit(train_x, train_y)

In [ ]:
test_pred_y = reg.predict(test_x)

In [ ]:
# Save the result into 'result.csv'

totalRows = test_pred_y.shape[0]
with open('result.csv', mode='w') as result_file:
    csv_writer = csv.writer(result_file, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    
    csv_writer.writerow(['dataid','prediction'])
    
    for i in range(totalRows):
        csv_writer.writerow([str(i+1),test_pred_y[i][0]])
    